<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF 335 - Tecnologías de Búsqueda en la Web 2019-II </h1>

<H3 align='center'> Laboratorio 2 - Motores de búsqueda clásicos </H3>
<hr style="height:2px;border:none"/>

<h4 align='left'> Alumnos: </h4>
<p align='left'> *Nombre*, *Rol* </p>
<p align='left'> *Nombre*, *Rol* </p>
<p align='left'> *Nombre*, *Rol* </p>
<hr style="height:2px;border:none"/>

<h1>Enunciado:</h1>

En este laboratorio desarrollarán una función de ranking clásica: Tf-idf y compararán los resultados contra la función Okapi-BM25.

A continuación se encuentra todo el código que permite implementar un motor de búsqueda clásico cuya función de ranking es BM25.

Una vez hayan terminado la implementación y guardado los resultados respondan las siguientes preguntas:

<b>¿En qué métrica se fijarían principalmente? ¿Cúal modelo tuvo mejores resultados? ¿Por qué creen que ocurrió esto?</b>

<h3>Rúbrica</h3>
<table>
    <tr>
    <th>Sección</th>
    <th>Puntaje</th>
  </tr>
  <tr> 
    <td>Tf-idf</td>
    <td>50</td>
  </tr>
  <tr> 
    <td>Gráficos</td>
    <td>20</td>
  </tr>
  <tr>
   <td>Comentarios y respuestas</td>
    <td>30</td>
  </tr>
</table>

Antes de empezar se deben importar todas las librerías y métodos que se utilizarán.

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
import nltk
import numpy as np
import operator
import re
import time

Las siguientes son variables globales que se usan para encontrar los documentos que actuarán como corpus en la experiencia.

El corpus escogido corresponde una pequeña fracción del dataset Tipster, en específico, se usarán las publicaciones de "Wall Street Jorunal" del año 1992. 

In [ ]:
path = "./WSJ92/"
path2= "./WSJ92_stemmed/"
archivo_nombres = "corpus_docs.txt"

La función preprocesamiento realiza la tokenización completa del texto que se le entregue, fue utilizada para procesar el corpus y para que no existan problemas al reconocer los tokens también se usará para parsear las consultas.

Como pueden observar el proceso es muy similar al que se realizó en la última sesión, pero se agregó una gran lista de caracteres a la lista de Stopwords.

In [ ]:
def preprocesamiento(texto, es_query = True):
    stoplist = stopwords.words('english')
    listastop=["'","...","''","#","<",">","}","{","]","[","´","?","¡","!","¿","/","*",
                '"',"-","--",";",":","::","=",")","(","&","|","'ve","'t","'s",
                "'ll","n't",">.<","<3","._.","~","<br","</","/>","<<",">>","~~",
                ",","d","na","m","''","´´","!!!!!",'"(','",','.;','[*','tm','").',
               "l",'"&',').*','://','www','http','"...',"=[", '@', '``']
    if(es_query):
        listastop.append(".")
    nuevostoplist= stoplist+listastop
    tokens = nltk.word_tokenize(texto)
    filtered = [token for token in tokens if token not in nuevostoplist]
    stemmed = []
    stemmer = SnowballStemmer("english")
    for word in filtered:
        stemmed.append(stemmer.stem(word))
    return stemmed

CorpusParser y QueryParser son dos clases que permitirán mantener el corpus y las consultas en memoria principal. Además QueryParser realiza el preprocesamiento recién definido a las consultas, no así para CorpusParser, ya que este procedimiento toma demasiado tiempo como para realizarlo cada vez que se necesite cargar el corpus.

In [ ]:
class CorpusParser:
    def __init__(self, filename="corpus_docs.txt"):
        self.filename = filename
        self.regex = re.compile('^#\s*\d+')
        self.corpus = dict()
        
    #Parseo de corpus adaptado a mis WSJ92_stemmed
    def parse(self):#Pasarle los nombres.txt
        f = open(self.filename,'r')
        blobs = []
        for s in f.readlines():
            s = s.strip()
            blobs.append(s)
        f.close()
        for docid in blobs:#for id in nombres...
            f_stemmed = open(path2+docid,'r')
            text=''
            for line in f_stemmed.readlines():
                line = line.strip()+' '
                text+=line
            self.corpus[docid] = nltk.word_tokenize(text) #dict[id] = contenido en tokens 
            f_stemmed.close()

    def get_corpus(self):
        return self.corpus


class QueryParser:
    def __init__(self, filename="querys.txt"):
        self.filename = filename
        self.queries = []

    def parse(self):
        file_querys =open(self.filename,'r')
        for line in file_querys.readlines():
            line = line.strip().split()
            line = preprocesamiento(' '.join(line))
            #preprocesar
            
            self.queries.append(line)
        file_querys.close()

    def get_queries(self):
        return self.queries

InvertedIndex será la clase que se encargue de crear el indice invertido que el motor requiere para poder llevar a cabo la recuperación de documentos por cada consulta que se ejecute. Para esta experiencia la recuperación será a través de la disyunción de los documentos que contengan cada palabra de la consulta, es decir, se recuperarán todos los documentos que presenten por lo menos una palabra de la consulta.

DocumentLengthTable se encargará de mantener en memoria principal el largo de los documentos, esto es muy útil para calcular eficientemente las funciones de ranking.

In [ ]:
class InvertedIndex:
    def __init__(self):
        self.index = dict()

    def __contains__(self, item):
        return item in self.index

    def __getitem__(self, item):
        return self.index[item]

    def add(self, word, docid):
        if word in self.index:
            if docid in self.index[word]:
                self.index[word][docid] += 1
            else:
                self.index[word][docid] = 1
        else:
            d = dict()
            d[docid] = 1
            self.index[word] = d

    #frequency of word in document
    def get_document_frequency(self, word, docid):
        if word in self.index:
            if docid in self.index[word]:
                return self.index[word][docid]
            else:
                raise LookupError('%s not in document %s' % (str(word), str(docid)))
        else:
            raise LookupError('%s not in index' % str(word))

    #frequency of word in index, i.e. number of documents that contain word
    def get_index_frequency(self, word):
        if word in self.index:
            return len(self.index[word])
        else:
            raise LookupError('%s not in index' % word)


class DocumentLengthTable:

    def __init__(self):
        self.table = dict()

    def __len__(self):
        return len(self.table)

    def add(self, docid, length):
        self.table[docid] = length

    def get_length(self, docid):
        if docid in self.table:
            return self.table[docid]
        else:
            raise LookupError('%s not found in table' % str(docid))

    def get_average_length(self):
        sum = 0
        for length in list(self.table.values()):
            sum += length
        return float(sum) / float(len(self.table))


def build_data_structures(corpus):
    idx = InvertedIndex()
    dlt = DocumentLengthTable()
    for docid in list(corpus.keys()):
        #build inverted index
        for word in corpus[docid]:
            idx.add(str(word), str(docid)) #agrega palabras al indice invertido
            
        #build document length table
        length = len(corpus[str(docid)])
        dlt.add(docid, length)
    return idx, dlt

score_BM25 es la función que se encarga de calcular el puntaje de ranking para cada documento recuperado, como pueden ver todos los valores utilizados están dentro de los estándares vistos en el curso.

In [ ]:
from math import log

k1 = 1.2
k2 = 100
b = 0.75
R = 0.0

def score_BM25(n, f, qf, r, N, dl, avdl):
    K = compute_K(dl, avdl)
    first = log( ( (r + 0.5) / (R - r + 0.5) ) / ( (n - r + 0.5) / (N - n - R + r + 0.5)) )
    second = ((k1 + 1) * f) / (K + f)
    third = ((k2+1) * qf) / (k2 + qf)
    return first * second * third

def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)) )

QueryProcessor es la clase que implementa las funciones de búsqueda, se le debe entregar el corpus sobre el cual se quieren buscar documentos y las consultas que se desean evaluar, para entregarselas se pueden utilizar las clases CorpusParser y QueryParser.

Como pueden ver en el código el parámetro rank de la función run corresponde a a la función que quieran utilizar, es aquí donde deben implementar la función faltante: score_tf_idf().

HINT: 

Recuerden que $f_{ij}$ es la frecuencia de una palabra $i$ en un documento $j$, que $N$ es la cantidad de documentos del corpus y que $df_{i}$ es la cantidad de documentos en los que aparece una palabra al menos una vez dentro del corpus. 
Además recuerden que la representación Tf-idf corresponde a vectores altamente dispersos donde cada palabra tiene una posición y el peso correspondiente a esa posición se calcula usando los términos $w_{ij}$ y $w_{iq}$ para documentos y consultas, respectivamente.

\begin{equation}
    \label{eq:tf}
    tf_{ij} = \frac{f_{ij}}{max\left(f_{1j},f_{2j},...,f_{|V|j} \right)}
\end{equation}

\begin{equation}
    \label{eq:tfidf}
     w_{ij}=tf_{ij}\times idf_{i}, \ idf_{i} = \log \frac{N}{df_{i}}
\end{equation}

\begin{equation}
     w_{iq}= \left(0.5 + 0.5 \cdot tf_{ij}\right) \times idf_{i}
\end{equation}

Por último usen la formula de similitud coseno para evaluar la cercanía entre ambos vectores.

\begin{equation}
Sim(q,d) = Cos(\overrightarrow{\rm V_q},\overrightarrow{\rm V_d}) = \frac{\overrightarrow{\rm V_q}^T \cdot \overrightarrow{\rm V_d}}{||\overrightarrow{\rm V_{q}} || \ ||\overrightarrow{\rm V_d}||}
\end{equation}

In [ ]:
class QueryProcessor:
    def __init__(self, queries, corpus):
        self.queries = queries
        self.index, self.dlt = build_data_structures(corpus)

    def run(self, rank='bm25'):
        results = []
        for query in self.queries:
            results.append(self.run_query(query[1:],rank))
        return results

    def run_query(self, query, rank):
        query_result = dict()
        for term in query:
            if term in self.index:
                doc_dict = self.index[term] # retrieve index entry
                for docid, freq in doc_dict.items(): #for each document and its word frequency
                    
                    if(rank=='bm25'):
                        score = score_BM25(n=len(doc_dict), f=freq, qf=1, r=0, N=len(self.dlt), dl=self.dlt.get_length(docid), avdl=self.dlt.get_average_length()) # calculate score
                    elif(rank=='tf-idf'):
                        score = score_tf_idf()#Esta es la funcion que deben implementar
                    else:
                        raise ValueError('rank must be bm25 or tf-idf.')
                    if docid in query_result: #this document has already been scored once
                        query_result[docid] += score
                    else:
                        query_result[docid] = score
        return query_result

Con la función guardar_resultados pueden almacenar los resultados que les entregue el motor de búsquedas: 

- results corresponde a los resultados del motor
- qids son las ids de las consultas, estas se manejan un poco más adelante en el código
- filename es el archivo destino en que se guardarán los resultados OJO: no sobre escriban sus resultados
- k es el largo del top ranking que desean guardar 



In [ ]:
def guardar_resultados(results, qids, filename, k=10, verbose=False):
    c_qid = 0
    with open(filename,"w") as file_resultados:
        for result in results:
            sorted_x = sorted(result.items(), key=operator.itemgetter(1))
            sorted_x.reverse()
            index = 0
            if(len(sorted_x)>k):
                for i in sorted_x[:k]:
                    tmp = (qids[c_qid], i[0], index, i[1])
                    el_string ='{:>1}\t{:>4}\t{:>2}\t{:>12}\tBM25'.format(*tmp)
                    file_resultados.write(el_string+'\n')
                    if(verbose):
                        print(el_string)
                    index += 1
            else:
                for i in sorted_x[:]:
                    tmp = (qids[c_qid], i[0], index, i[1])
                    el_string ='{:>1}\t{:>4}\t{:>2}\t{:>12}\tBM25'.format(*tmp)
                    file_resultados.write(el_string+'\n')
                    if(verbose):
                        print(el_string)
                    index += 1
            c_qid += 1

La función read_qrels encontrará las listas de relevancias para cada consulta que utilizaremos.

In [ ]:
def read_qrels(filename="qrels.txt"):
    rels = dict()
    with open(filename,'r') as f_qrels:
        for linea in f_qrels.readlines():
            linea = linea.strip().split()
            llave_r = linea[0]
            docid_r = linea[2]
            if(int(linea[-1]) == 1):
                if(llave_r in list(rels.keys())):
                    rels[llave_r].append(docid_r)
                else:
                    rels[llave_r] = [docid_r]
    return rels

En la siguiente celda se define una función que calculará la Mean Precision (MP), Average Recall hasta la posición k (AR@k) y el Normalized Discounted Cumulative Gain hasta la posición k (NDCG@k).

In [ ]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

def calcular_metricas(files, rels, k = 10, verbose=False): 
    with open(files,'r') as f_resultados:
        resultados = dict()
        for line in f_resultados.readlines():
            line = line.strip().split()
            llave = line[0]
            docid_path = line[1].split('/')
            docid = docid_path[-1]
            if(llave in list(resultados.keys())):
                resultados[llave].append(docid)
            else:
                resultados[llave] = [docid]
    avg_p_list=[]
    avg_r_list=[]
    avg_ndcg_list=[]
    for llave in list(resultados.keys()):
        rel_llave = llave
        if(llave[0]=='0'):
            rel_llave = llave[1:]
        relevantes = set(rels[rel_llave])
        precision_list = []
        recall_list = []
        rel_bool_list =[]
        for i in range(1,len(resultados[llave])+1):
            rel_doc = resultados[llave][i-1]
            recuperados = set(resultados[llave][:i])
            inter = relevantes & recuperados
            if(rel_doc in relevantes):
                precision = 100*float(len(inter))/i
                precision_list.append(precision)
                rel_bool_list.append(1)
            else:
                rel_bool_list.append(0)
            if(i == k):
                recall = 100*float(len(inter))/len(relevantes)
                recall_list.append(recall)
                break

        ndcg_query = ndcg_at_k(rel_bool_list, k,method=1)
        if(len(precision_list)!=0):
            avg_p = sum(precision_list)/len(precision_list)
        else:
            avg_p = 0.0
        if(len(recall_list)!=0):
            avg_r = sum(recall_list)/len(recall_list)

        avg_p_list.append(avg_p)
        avg_r_list.append(avg_r)
        avg_ndcg_list.append(ndcg_query)
        if(verbose):
            print('***********************************')
            print("Query "+str(llave))
            print("@"+str(k))
            print("Avg. Precision: {:.4f}%".format(avg_p))
            print("Avg. Recall: {:.4f}%".format(avg_r))
            print("NDCG: {:.4f}%".format(ndcg_query))
    avg_p_modelo = sum(avg_p_list)/len(avg_p_list)
    avg_r_modelo = sum(avg_r_list)/len(avg_r_list)
    avg_ndcg_modelo = sum(avg_ndcg_list)/len(avg_ndcg_list)
    print('----------------------------------------------')
    print("TOTAL MODELO: "+files)
    print("@"+str(k))
    print("Avg. Precision: {:.4f}%".format(avg_p_modelo))
    print("Avg. Recall: {:.4f}%".format(avg_r_modelo))
    print("Avg. NDCG: {:.4f}%".format(avg_ndcg_modelo))
    return avg_p_list, avg_r_list, avg_ndcg_list

Teniendo todo definido utilizaremos las clases para probar el modelo BM25:

In [ ]:
cp = CorpusParser()
cp.parse()
corpus = cp.get_corpus()
print("Corpus parceado")

In [ ]:
parser = QueryParser()
parser.parse()
queries = parser.get_queries()
print("Queries parseadas")

La siguiente celda sirve para rescatar las IDs de las consultas en una lista:

In [ ]:
qids =[]
for i in range(len(queries)):
        q = queries[i]
        qids.append(q[0])

In [ ]:
proc = QueryProcessor(queries, corpus)
#La linea anterior puede tomar un poco de tiempo,
#Ya que esta armando el indice invertido

start = time.time()
results = proc.run() #para usar tf-idf deben colocar rank='tf-idf'
print('Tiempo (s): {}'.format(time.time() - start))
guardar_resultados(results,qids,filename="resultados_BM25.txt",k=50)

In [ ]:
rels = read_qrels()
files = "resultados_BM25.txt"

x = range(1,51)

mean_l_p = []
var_l_p = []

mean_l_r = []
var_l_r = []

mean_l_ndcg = []
var_l_ndcg =[]

for i in x:
    
    avg_p_list, avg_r_list, avg_ndcg_list = calcular_metricas(files, rels, k=i)
    
    avg_p_modelo = sum(avg_p_list)/len(avg_p_list)
    mean_l_p.append(avg_p_modelo)
    var_l_p.append(np.std(avg_p_list))
    
    avg_r_modelo = sum(avg_r_list)/len(avg_r_list)
    mean_l_r.append(avg_r_modelo)
    var_l_r.append(np.std(avg_r_list))
    
    avg_ndcg_modelo = sum(avg_ndcg_list)/len(avg_ndcg_list)
    mean_l_ndcg.append(avg_ndcg_modelo)
    var_l_ndcg.append(np.std(avg_ndcg_list))



Repitan estos gráficos para los resultados obtenidos de tf-idf y comparen.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

plt.plot(x, mean_l_p, 'o',color='blue')
plt.plot(x, mean_l_p, '-', color='blue')
plt.title("Curva de Precision")
plt.xlabel("K")
plt.show()

plt.plot(x, mean_l_r, 'or')
plt.plot(x, mean_l_r, '-', color='red')
plt.title("Curva de Recall")
plt.xlabel("K")
plt.show()

plt.plot(x, mean_l_ndcg, 'o',color='green')
plt.plot(x, mean_l_ndcg, '-', color='green')
plt.title("Curva de Recall")
plt.xlabel("K")
plt.show()
